In [ ]:
!pip install telebot
import telebot
import shutil


import os
import telebot
from telebot import types
BOT_TOKEN = '6568653907:AAElgwsasILjTUDDZKoOJFeSBbqEqB3xW6s'



bot = telebot.TeleBot(BOT_TOKEN)
nlp_model = pipeline("question-answering", model=model, tokenizer=tokenizer)
def categorize_prediction(probability):
    if probability < 0.3:
        return "low"
    elif 0.3 <= probability < 0.7:
        return "moderate"
    else:
        return "high"

@bot.message_handler(content_types=['photo'])
def echo_photo(message):
    # Get the photo ID of the largest available photo
    photo = message.photo[-1]

    # Download the photo
    photo_info = bot.get_file(photo.file_id)
    photo_url = f"https://api.telegram.org/file/bot{BOT_TOKEN}/{photo_info.file_path}"

    # Define the folder where images will be saved
    save_folder = "saved_images"
    try:
        shutil.rmtree("/content/saved_images")
        shutil.rmtree("/content/runs/detect/predict")

        #print(f"Directory '{directory_path}' and its contents have been removed.")
    except Exception as e:
        print(f"Error while removing directory: {e}")

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Define the image filename
    image_filename = f"{message.from_user.id}_{message.message_id}.jpg"
    image_path = os.path.join(save_folder, image_filename)

    os.system(f"wget -O {image_path} {photo_url}")
    !yolo task=detect mode=predict model=/content/drive/MyDrive/cancer.pt conf=0.25 source='/content/saved_images' save=True
    # Send the downloaded photo back to the user
    final_image_path = os.path.join("/content/runs/detect/predict/", image_filename)

    with open(final_image_path, 'rb') as photo_file:
        print(image_path)
        bot.send_photo(message.chat.id, photo_file)

    bot.reply_to(message, "Image saved and sent!")
     # Extract text from the processed image using OCR
    text = pytesseract.image_to_string(final_image_path)
    prediction_lines = text.split('\n')
    responses = []
    for prediction_text in prediction_lines:
         if prediction_text.strip():
            result = nlp_model(question="What is this prediction?", context=prediction_text)

            # Extract the predicted answer and probability for this prediction
            predicted_answer = result['answer']
            prediction_probability = result['score']

            # Categorize the prediction based on probability using your categorize_prediction function
            prediction_category = categorize_prediction(prediction_probability)

            # Define a response
            response = f"Prediction: {prediction_text}\nNLP Answer: {predicted_answer}\nPrediction Category: {prediction_category}"
            responses.append(response)
     # Send the responses for each prediction
    for response in responses:
        bot.send_message(message.chat.id, response)
    # Send the downloaded photo back to the user
    with open(final_image_path, 'rb') as photo_file:
        bot.send_photo(message.chat.id, photo_file)

    # Send the NLP response to the user
    bot.reply_to(message, response)


@bot.message_handler(func=lambda message: True)
def handle_text_messages(message):
    bot.reply_to(message, "Please send an image.")

if _name_ == "_main_":
    bot.polling(none_stop=True)

ERROR: Could not find a version that satisfies the requirement some_module (from versions: none)
ERROR: No matching distribution found for some_module


NameError: ignored